In [2]:
import ee
import os
ee.Initialize()

### Make a folder to hold fuel layers generated from your treatment scenario ensemble

In [3]:
scenarios_path = "projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/treatment_scenarios" # change this to given runs folder path
scenarios = ee.ImageCollection(scenarios_path)
#create a folder representing scenarios imgColl name scheme
fuels_folder=scenarios_path.replace('treatment_scenarios','fuelscapes_scenarios')
fuels_folder_list = os.popen(f"earthengine ls {os.path.dirname(scenarios_path)}").read().split('\n')[0:-1]
if not fuels_folder in fuels_folder_list:
    os.popen(f"earthengine create folder {fuels_folder}").read()
    print(f'Created Folder: {fuels_folder}')
else:
    print(f"{fuels_folder} already exists")

projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios already exists


### For each treated landscape img in your treatment scenarios imageCollection, create a new folder to hold the fuels layers

In [4]:
scenarios_img_paths = os.popen(f"earthengine ls {scenarios_path}").read().split('\n')[0:-1]
scn_sub_folders = [(fuels_folder+'/'+os.path.basename(path)) for path in scenarios_img_paths]

sub_folders_list = os.popen(f"earthengine ls {fuels_folder}").read().split('\n')[0:-1]
for sub_folder_name in scn_sub_folders:
    if not sub_folder_name in sub_folders_list:
        os.popen(f"earthengine create folder {sub_folder_name}").read()
        print(f'Created scenario fuelscapes sub-folder: {sub_folder_name}')
    else:
        print(f"{sub_folder_name} already exists")

projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario1 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario10 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario11 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario12 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario2 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario3 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario4 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenario

### Loop thru each fuel updater script the way we do for FireFactor: create_canopy_guide, calc_CC_CH, calc_CBD_CBH, calc_FM40 (don't need to run fm40 wui update)

In [5]:
# get config.yml file path, needed for every script
config_path = f"{os.getcwd()}\config.yml" # on windows its a forward slash

### You can run Canopy Guide and FM40 updater scripts at same time since they do not depend on one another

In [ ]:
# Canopy Guide
# for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
#     cmd = f"python src/create_canopy_guide.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
#     print(cmd)
#     os.popen(cmd).read()
#     #break

# FM40
# for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
#     cmd = f"python src/calc_FM40.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
#     print(cmd)
#     os.popen(cmd).read()
#     #break


### You must wait until all Canopy Guide exports have successfully completed before running the CC_CH script

In [ ]:
# CC and CH
# for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
#     cmd = f"python src/calc_CC_CH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
#     print(cmd)
#     os.popen(cmd).read()
#     print('\n')
#     #break

### wait until all CC and CH exports have completed before running CBD_CBH 

In [5]:
# CBD and CBH
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cmd = f"python src/calc_CBD_CBH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    print('\n')
    break

python src/calc_CBD_CBH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/treatment_scenarios/scenario1 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario1




### After all fuel layers in each scenario's fuelscape has successfully exported, collate each 5-layer fuelscape into a multiband img for Google Drive Export

In [6]:
# must mosaic the canopy guide and fm40 collections (in this case just .first()) 
# since they are exported as zone-wise img collections by default
import yml_params
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    #print(scn_img_path)
    print(scn_sub_folder)
    fm40 = ee.ImageCollection(scn_sub_folder+'/fm40_collection').select('new_fbfm40').mosaic()
    cc = ee.Image(scn_sub_folder+'/CC')
    ch = ee.Image(scn_sub_folder+'/CH')
    #cbh = ee.Image(scn_sub_folder+'/CBH')
    #cbd = ee.Image(scn_sub_folder+'/CBD')
    fuel_stack = fm40.addBands(cc).addBands(ch)
    print(fuel_stack.bandNames())

    # export
    runs_id = scn_sub_folder.split('/')[6]
    scn_id = scn_sub_folder.split('/')[-1]
    print(runs_id)
    print(scn_id)
    
    desc = f"export_{runs_id}_{scn_id}_fuelStack"
    crs,scale = yml_params.get_export_params(config_path)
    print(desc)
    print(crs)
    print(scale)
    folder='AFF_Fuelscapes'
    fileNamePrefix=f'{runs_id}/fuelStack_{scn_id}'
    
    task = ee.batch.Export.image.toDrive(image=fuel_stack,description=desc,folder=folder,fileNamePrefix=fileNamePrefix,region=fuel_stack.geometry(),scale=scale,crs=crs)
    task.start()
    print(f'Export started: {folder}/{fileNamePrefix}')
    break

projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/treatment_scenarios/scenario1
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario1
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/treatment_scenarios/scenario10
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario10
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/treatment_scenarios/scenario11
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario11
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/treatment_scenarios/scenario12
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/fuelscapes_scenarios/scenario12
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_12_distro_log_20220811/treatment_sc